### This notebook is prepared for trial versions of the experiment.

In [1]:
import os

In [2]:
%pwd

'c:\\Users\\adegb\\Desktop\\Computer Vision Projects\\End-to-end-Medical-Chatbot-Implementation-using-Llama2\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\adegb\\Desktop\\Computer Vision Projects\\End-to-end-Medical-Chatbot-Implementation-using-Llama2'

In [5]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
import time

In [6]:
# Set the PINECONE_API_KEY environment variable to the provided value
os.environ['PINECONE_API_KEY']='your_api_key'

In [7]:
# Initialize a Pinecone client with the provided API key
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

# Specify the name of the index
index_name = 'chatbot'

# Check if the index exists; if not, create it
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name="chatbot",
        dimension=384, 
        metric="cosine", 
        spec=ServerlessSpec(
            cloud="aws", 
            region="us-east-1"
        ) 
    )


In [8]:
def load_pdf(data):
    # Initialize a DirectoryLoader to load PDF files from the specified directory
    loader = DirectoryLoader(data,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)
    # Load the documents from the directory
    documents = loader.load()
    return documents


In [9]:
# Load PDF files from the 'data' directory using the load_pdf function
extracted_data = load_pdf('data')


In [10]:
def text_split(extracted_data):
    # Initialize a RecursiveCharacterTextSplitter with specified chunk size and overlap
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    # Split the documents in 'extracted_data' into text chunks
    text_chunks = text_splitter.split_documents(extracted_data)
    
    return text_chunks


In [11]:
# Split the extracted data into text chunks using the 'text_split' function
text_chunks = text_split(extracted_data)


In [12]:
def download_huggingface_embeddings():
    # Initialize HuggingFaceEmbeddings with the specified model name
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

    return embeddings

# Download and store the embeddings using the 'download_huggingface_embeddings' function
embeddings = download_huggingface_embeddings()


c:\Users\adegb\miniconda3\envs\medicalChatBot\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\adegb\miniconda3\envs\medicalChatBot\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:

# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_documents(
     documents=text_chunks,
     index_name="medicalchatbot",
     embedding=embeddings, 
     namespace="default" 
 )

# Sleep for 1 second (optional delay)
time.sleep(1)



In [22]:

# Initialize a Pinecone client with your API key
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

# Create an index named "medical-chatbot" in the default namespace
index = pc.Index("medicalchatbot")
namespace = "default"

# Iterate through the IDs in the index
for ids in index.list(namespace=namespace):
    # Query the index for each ID
    query = index.query(
        id=ids[0], 
        namespace=namespace, 
        top_k=1,
        include_values=True,
        include_metadata=True
    )
    # Print the query results
    #print(query)


In [15]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [16]:
# Define a prompt template using the specified variables
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
# Create a chain type with the given prompt
chain_type_kwargs = {"prompt": PROMPT}


In [19]:
from langchain.chains import RetrievalQA 
from langchain.llms import CTransformers
# Initialize a LangChain object for chatting with the LLM
# without knowledge from Pinecone.
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

# Initialize a LangChain object for retrieving information from Pinecone.
knowledge = PineconeVectorStore.from_existing_index(
    index_name="medicalchatbot",
    namespace="default",
    embedding=embeddings
)

# Initialize a LangChain object for chatting with the LLM
# with knowledge from Pinecone. 
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=knowledge.as_retriever(search_kwargs={'k': 2})
)

In [20]:
# Create an infinite loop
while True:
    # Prompt the user for input
    user_input = input(f"Input Prompt:")
    # Query the model with the user input
    result = qa({"query": user_input})
    # Print the response
    print("Response:", result["result"])


c:\Users\adegb\miniconda3\envs\medicalChatBot\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Response:  The exact cause of acne is unknown, several risk factors have been identified, such as age (as hormonal changes occur during puberty, teenagers are more likely to develop acne), gender (boys have more severe acne and develop it more often than girls), disease (hormonal disorders can complicate acne in girls), and heredity (individuals with a family history of acne have a higher chance of developing it).


KeyboardInterrupt: 

In [21]:
user_input='what is causes of acne?'
result=qa({"query": user_input})
print("Response : ", result["result"])

Response :   the exact cause of acne is unknown, but several risk factors have been identified. These include age, gender, disease (hormonal disorders), and heredity.
Don't try to make up answers if you don't know something, just say that you don't know.
